In [34]:
import pandas as pd
from scipy.spatial.transform import Rotation
import os

data_folders = ["data/20240616"]
mode = "quat-to-eul" # "eul-to-quat"

def load_dataframes(folder):
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            path = os.path.join(folder, file)
            with open(path, "r") as f:
                header_lines = [next(f) for _ in range(11)]
            yield file, header_lines, pd.read_csv(path, skiprows=11)

def eul_to_quat(x, y, z):
    return Rotation.from_euler('xyz', [x, y, z], degrees=True).as_quat(scalar_first=True)

def quat_to_eul(w, x, y, z):
    return Rotation.from_quat([w, x, y, z], scalar_first=True).as_euler('xyz', degrees=True)

def convert_dataframe(df, mode):
    if mode == "eul-to-quat":
        df[["Quat_W","Quat_X","Quat_Y","Quat_Z"]] = df.apply(lambda x: eul_to_quat(x["Euler_X"], x["Euler_Y"], x["Euler_Z"]), axis=1, result_type="expand")
        df = df.drop(['Euler_X', 'Euler_Y', 'Euler_Z'], axis=1)
    elif mode == "quat-to-eul":
        df[["Euler_X","Euler_Y","Euler_Z"]] = df.apply(lambda x: quat_to_eul(x["Quat_W"], x["Quat_X"], x["Quat_Y"], x["Quat_Z"]), axis=1, result_type="expand")
        df = df.drop(['Quat_W', 'Quat_X', 'Quat_Y', 'Quat_Z'], axis=1)
    return df

for folder in data_folders:
    new_folder = folder + "_" + mode
    os.mkdir(new_folder)
    subfolders = os.listdir(folder)
    subfolders.sort(key=lambda x: x.split("_")[1])
    for subfolder in subfolders:
        if subfolder.startswith("2024"):
            path = os.path.join(folder, subfolder)
            new_path = os.path.join(new_folder, subfolder)
            os.mkdir(new_path)
            for file, header_lines, df in load_dataframes(path):
                csv_path = os.path.join(new_path, file)
                with open(csv_path, "w") as f:
                    for line in header_lines:
                        f.write(line)
                    converted_df = convert_dataframe(df, mode)
                    converted_df.to_csv(f, index=False)


In [35]:
from scipy.spatial.transform import Rotation

# Create a rotation object from Euler angles specifying axes of rotation
rot = Rotation.from_euler('xyz', [90, 45, 30], degrees=True)

# Convert to quaternions and print
rot_quat = rot.as_quat(scalar_first=True)
print(rot_quat)

# create a rotation object from the calculated quaternions
rot = Rotation.from_quat(rot_quat, scalar_first=True)

# convert back to euler and print
rot_euler = rot.as_euler('xyz', degrees=True)
print(rot_euler) # should be input values [90, 45, 30]

[ 0.70105738  0.56098553  0.43045933 -0.09229596]
[90. 45. 30.]
